In [45]:
import requests
from bs4 import BeautifulSoup
import re
import csv
import  modelswb

In [50]:
class Parse:
    def __init__(self, url:str):
        self.brand_id = self.__get_id_brand(url)
    
    @staticmethod
    def __get_id_brand(url:str):
        brand_id = re.search(r'(?:\?brandpage=)(.+?)(?:__)', url).group(1)
        return brand_id
    
    def request_(self):
        self.__create_csv()
        page = 1
        while True:
            headers = {
                'Accept': '*/*',
                'Accept-Language': 'en-US,en;q=0.9,ru;q=0.8',
                'Connection': 'keep-alive',
                'DNT': '1',
                'Origin': 'https://www.wildberries.by',
                'Referer': 'https://www.wildberries.by/',
                'Sec-Fetch-Dest': 'empty',
                'Sec-Fetch-Mode': 'cors',
                'Sec-Fetch-Site': 'cross-site',
                'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
                'sec-ch-ua': '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
                'sec-ch-ua-mobile': '?0',
                'sec-ch-ua-platform': '"Linux"',
                }
            response = requests.get(
                f'https://catalog.wb.ru/brands/m/catalog?brand={self.brand_id}&limit=100&sort=popular&page={page}&appType=128&curr=byn&locale=by&lang=ru&dest=-59208&regions=1,4,22,30,31,33,40,48,66,68,69,70,80,83,114&reg=1&spp=0',
                headers=headers)
            page += 1
            box = modelswb.Main_box.parse_obj(response.json()['data'])
            if not box.products:
                break
            self.__save_scv(box)
        
    def __create_csv(self):
        with open('parsing_file.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['id', 'name', 'price', 'brand', 'sales_count', 'raiting', 'have_now'])
    
    def __save_scv(self, items):
        with open('parsing_file.csv', 'a', newline='') as file:
            writer = csv.writer(file)
            for i in items.products:
                writer.writerow([i.id,
                                 i.name,
                                 i.salePriceU,
                                 i.brand,
                                 i.sale,
                                 i.rating,
                                 i.volume
                                 ])

In [51]:
x = Parse('https://www.wildberries.by/catalog?brandpage=27445__MSI')
print(x.brand_id)
print(x.request_())

27445
None
